In [7]:
from bs4 import BeautifulSoup
import requests
import re
import xlwt
from xlrd import open_workbook

restaurant_dict = {}
basicinfo = [['編號', '餐廳名稱', '地址', '平均消費', '餐廳類型']]
detailinfo = [['餐廳簡介', '餐廳電話', '鄰近捷運站', '餐廳補充說明', '推薦菜色', '餐廳評分', '週一', '週二', '週三', '週四', '週五',
              '週六', '週日', '營業時間', '圖片網址一', '圖片網址二', '圖片網址三', '圖片網址四', '圖片網址五']]
url_list = []
pic_list = []
i = 1
c = 1
for page in range(1499, 1500):
    #取出搜尋頁面之店家基本資料
    ipeen_url = 'http://www.ipeen.com.tw/search/all/000/1-0-0-0/?p=' + str(page) + '&adkw=%E5%8F%B0%E5%8C%97'
    res = requests.get(ipeen_url)
    soup = BeautifulSoup(res.text, "lxml")
    for item in soup.select('.serItem'):
        #將店名以dict的方式儲存地址，去除店名&地址都一樣的店
         if item.select('.ga_tracking')[0].text not in restaurant_dict or item.select('.address')[0].text.strip() != restaurant_dict[item.select('.ga_tracking')[0].text]:
            try:    
                restaurant_dict[item.select('.ga_tracking')[0].text] = item.select('.address')[0].text.strip()
                #第1~5欄:  1.編號  2.餐廳名稱  3.地址  4.平均消費  5.餐廳類型
                z1 = item.select('.ga_tracking')[0].text         #餐廳名稱
                z2 = item.select('.address')[0].text.strip()     #地址
                z3 = item.select('.costEmpty')[0].text.strip()   #平均消費
                z4 = item.select('.cate')[0].text.strip()        #餐廳類型
                save1 = [c, z1, z2, z3, z4]
                c += 1
                basicinfo.append(save1)
            except IndexError:
                pass
    #取出該店家之網址以及餐廳圖片網址  
    link_list =re.findall(r"(?<=href=\").+?(?=\")|(?<=href=\').+?(?=\')" , res.text)
    for url in link_list:
        if '/shop/' in url and 'discount' not in url:
            fixurl = 'http://www.ipeen.com.tw' + url
            if fixurl not in url_list:
                picurl = fixurl[:fixurl.find('-')] + '/photos'
                url_list.append(fixurl) #將餐廳網址寫入url_list
                pic_list.append(picurl) #將餐廳圖片網址寫入pic_list
                
                print(fixurl, page)
                
                #取得店家詳細資料
                s_res = requests.get(fixurl)
                s_res.encoding = 'utf-8'
                s_soup = BeautifulSoup(s_res.text, "lxml")
                k = s_soup.select('.clearfix')
                if len(k) < 2:
                    z5 = z6 = z7 = z8 = z9 = z10 = t1 = t2 = t3 = t4 = t5 = t6 = t7 = t8 = 'N/A'
                else:
                    #第6~11欄:   6.餐廳簡介  7.餐廳電話  8.鄰近捷運站  9.餐廳補充說明  10.推薦菜色  11.餐廳評分
                    try:
                        z5 = k[1].select(".summary")[0].text.strip().replace(u'\r\n', u'')     #餐廳簡介
                    except IndexError:
                        z5 = ''
                    if k[2].find("th", text="電話") != None:
                        z6 = k[2].find("th", text="電話").find_next_sibling("td").text.strip() #餐廳電話
                    else:
                        z6 = ''
                    z7 = k[2].find("th", text="捷運資訊").find_next_sibling("td").text.strip().replace(u'\xa0', u' ')  #鄰近捷運站
                    #補充說明資料
                    if k[2].find("th", text="補充說明") != None:
                        z8 = k[2].find("th", text="補充說明").find_next_sibling("td").text.strip()
                    else:
                        z8 = ''
                    z9 = k[2].select('.recommend')[0].text.strip().replace(u'\n\n', u':').replace(u'\u3000', u' ')  #推薦菜色
                    #餐廳評分
                    h = s_soup.select('#shop-header')
                    z10 = h[0].select('.scalar')[0].select('.rating')[0].text.strip().replace(u'\r\n', u'').replace(u' ', u'').replace(u'\n', u'')
                    #營業時間
                    if k[2].find("td", text="週一") != None and k[2].find("td", text="週二") != None:
                        if k[2].find("td", text="週一").find_next_sibling("td") != None:
                            t1 = k[2].find("td", text="週一").find_next_sibling("td").text.strip().replace(u'\xa0', u' ')
                        else:
                            t1 = ''
                        if k[2].find("td", text="週二").find_next_sibling("td") != None:
                            t2 = k[2].find("td", text="週二").find_next_sibling("td").text.strip().replace(u'\xa0', u' ')
                        else:
                            t2 = ''
                        if k[2].find("td", text="週三").find_next_sibling("td") != None:
                            t3 = k[2].find("td", text="週三").find_next_sibling("td").text.strip().replace(u'\xa0', u' ')
                        else:
                            t3 = ''
                        if k[2].find("td", text="週四").find_next_sibling("td") != None:
                            t4 = k[2].find("td", text="週四").find_next_sibling("td").text.strip().replace(u'\xa0', u' ')
                        else:
                            t4 = ''
                        if k[2].find("td", text="週五").find_next_sibling("td") != None:
                            t5 = k[2].find("td", text="週五").find_next_sibling("td").text.strip().replace(u'\xa0', u' ')
                        else:
                            t5 = ''
                        if k[2].find("td", text="週六").find_next_sibling("td") != None:
                            t6 = k[2].find("td", text="週六").find_next_sibling("td").text.strip().replace(u'\xa0', u' ')
                        else:
                            t6 = ''
                        if k[2].find("td", text="週日").find_next_sibling("td") != None:
                            t7 = k[2].find("td", text="週日").find_next_sibling("td").text.strip().replace(u'\xa0', u' ')
                        else:
                            t7 = ''
                        t8 = ''
                    elif k[2].find("th", text="營業時間") != None:
                        t8 = k[2].find("th", text="營業時間").find_next_sibling("td").text.strip()
                        t1 = t2 = t3 = t4 = t5 = t6 = t7 = ''
                    else:
                        t1 = t2 = t3 = t4 = t5 = t6 = t7 = t8 = ''
                save2 = [z5, z6, z7, z8, z9, z10, t1, t2, t3, t4, t5, t6, t7, t8]
                #餐廳照片位址
                p_res = requests.get(picurl)
                p_res.encoding = 'utf-8'
                p_soup = BeautifulSoup(p_res.text, "lxml")
                kp =p_soup.select('#review-photo')
                try:
                    list_all_pic = kp[0].find_all("a")
                except IndexError:
                    kp =p_soup.select('#vip-photo')
                    try:
                        list_all_pic = kp[0].find_all("a")
                    except IndexError:
                        try:
                            kp =p_soup.select('#liveshare-photo')
                            list_all_pic = kp[0].find_all("a")
                        except IndexError:
                            list_all_pic = []
                for num in range(0,5):
                    if num < len(list_all_pic):
                        save2.append(list_all_pic[num].get('href'))
                    else:
                        save2.append('')
                detailinfo.append(save2)
book = xlwt.Workbook(encoding="utf-8")#新建excel文檔
sheet = book.add_sheet('sheet1') #新建sheet
row = 0  # 循環寫列
#寫入excel檔
for basic in basicinfo:
    col = 0
    for bs in basic:
        sheet.write(row,col,bs)
        col+=1
    row+=1
#寫入excel檔
rowd = 0
for detail in detailinfo:
    col = 5
    for dt in detail:
        sheet.write(rowd,col,dt)
        col+=1
    rowd+=1
book.save('basicinfo.xls') #保存名稱

http://www.ipeen.com.tw/shop/1256274-蘑菇森林義式廚房 1499
http://www.ipeen.com.tw/shop/1308270-品火鍋 1499
http://www.ipeen.com.tw/shop/1308286-諭之園 1499
http://www.ipeen.com.tw/shop/1308295-敲敲咖啡 1499
http://www.ipeen.com.tw/shop/1308301-偷嘴甜點手創室 1499
http://www.ipeen.com.tw/shop/1308309-STARBUCKS-統一星巴克-南京復興門市 1499
http://www.ipeen.com.tw/shop/1308315-彼得好咖啡 1499
http://www.ipeen.com.tw/shop/1308317-慢食堂-LU·KITCHEN 1499
http://www.ipeen.com.tw/shop/1308330-品川蘭-中山店 1499
http://www.ipeen.com.tw/shop/1308339-小仺屋Cafe-Rice 1499
http://www.ipeen.com.tw/shop/1308350-Sidebar 1499
http://www.ipeen.com.tw/shop/1308362-慕門咖啡 1499
http://www.ipeen.com.tw/shop/1308375-無敵搶鍋麵專賣 1499
http://www.ipeen.com.tw/shop/1308376-Steam-E-den-Café-蒸氣龐克復古咖啡廳 1499
http://www.ipeen.com.tw/shop/1308387-莫凡彼-台北天母店 1499
http://www.ipeen.com.tw/shop/1308407-澄朵Brunch 1499
